Updated notebook - main changes: revert to separating each category into its own collection, implement 2 step pipeline (categorize, then retrieve from those relevant collections). Somehow, takes a substantial amount of time now (ie 7 queries in 17 minutes) (using general qa set and some other random set of data), to discuss.

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub
!pip install llama-cpp-python
!pip install langchain-community langchain-core
!pip install evaluate
!pip install bert_score

Define Variables

In [ ]:
from llama_cpp import Llama

model_path = "mistral-7b-instruct-v0.2.Q4_0.gguf"
model = Llama(model_path=model_path, n_ctx=2048, n_threads=8, verbose=False)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")


Define Data Sources

In [ ]:
import pandas as pd
import concurrent.futures
import uuid
import os

file_names = [
    "study_permit_general", "work_permit_student_general", "work-study-data-llm",
    "vancouver_transit_qa_pairs", "permanent_residence_student_general", "data-with-sources",
    "faq_qa_pairs_general", "hikes_qa", "sfu-faq-with-sources", "sfu-housing-with-sources",
    "sfu-immigration-faq", "park_qa_pairs-up", "cultural_space_qa_pairs_up",
    "qa_pairs_food", "qa_pairs_year_and_month_avg", "qa_pairs_sfu_clubs"
]

collections = {}
batch_size = 32

def process_file(file):
    try:
        path = f'../Data/{file}.csv'
        if not os.path.exists(path):
            return f"{file} skipped (file not found)."

        df = pd.read_csv(path, usecols=lambda col: col.lower() in {"question", "answer"})
        df.columns = df.columns.str.lower()

        if "question" not in df.columns or "answer" not in df.columns:
            return f"{file} skipped (missing question/answer columns)."

        df = df.drop_duplicates(subset="question")
        df["text"] = df["question"].fillna('') + ' ' + df["answer"].fillna('')
        unique_texts = list(set(df["text"].dropna().tolist()))

        collection = client.get_or_create_collection(name=file)
        for i in range(0, len(unique_texts), batch_size):
            batch = unique_texts[i:i + batch_size]
            embeddings = embedding_model.embed_documents(batch)
            ids = [str(uuid.uuid4()) for _ in batch]
            collection.add(ids=ids, embeddings=embeddings, documents=batch)

        collections[file] = collection
        return f"{file}: Loaded {len(unique_texts)} docs."
    except Exception as e:
        return f"{file}: Error - {e}"

# parallelogram
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    results = list(executor.map(process_file, file_names))

for result in results:
    print(result)


In [11]:
# define cat to collection mapping
# motivation: takes wayyyy too long now -> dataset size trippled and time grew exponentially...
# now takes around 2 mins on avg for each response..compared to 40 seconds previously..

collection_map = {
    "study_permit": "study_permit_general",
    "work_permit": "work_permit_student_general",
    "work_study": "work-study-data-llm",
    "public_transit": "vancouver_transit_qa_pairs",
    "permanent_residence": "permanent_residence_student_general",
    "health_related": "data-with-sources",
    "general_faqs": "faq_qa_pairs_general",
    "vancouver_hiking": "hikes_qa",
    "university_general_faqs": "sfu-faq-with-sources",
    "university_housing": "sfu-housing-with-sources",
    "university_immigration_faqs": "sfu-immigration-faq",
    "vancouver_parks": "park_qa_pairs-up",
    "vancouver_cultural": "cultural_space_qa_pairs_up",
    "university_food": "qa_pairs_food",
    "expenditure": "qa_pairs_year_and_month_avg",
    "university_clubs": "qa_pairs_sfu_clubs"
}


Function to now match for releveant document

In [5]:
def get_relevant_documents(query, categories, n_results=3):
    all_results = []
    query_embedding = embedding_model.embed_documents([query])[0]

    for category in categories:
        collection_name = collection_map[category]
        if collection_name in collections:
            try:
                result = collections[collection_name].query(
                    query_embeddings=[query_embedding],
                    n_results=n_results
                )
                docs = result.get("documents", [[]])[0]
                sims = result.get("distances", [[]])[0]

                all_results.extend(zip(docs, sims))
            except Exception as e:
                print(f"error querying {collection_name}: {e}")

    all_results = sorted(all_results, key=lambda x: x[1])

    return all_results[:n_results]


### Classify Prompt

In [6]:
import re
import difflib

valid_categories = list(collection_map.keys())
fallback_category = "faq"

def classify_query(query):
    category_prompt = f"""
    You are a classifier for a Q&A system for international students in British Columbia.
    Pick ONLY from this list of category names (copy them exactly, case-insensitive), and return up to 3 relevant ones (comma-separated):

    {", ".join(valid_categories)}

    Query: "{query}"

    Return only the category name(s) as a comma-separated string.
    """

    response = model(category_prompt, max_tokens=50, temperature=0.1)["choices"][0]["text"].strip().lower()
    print("Raw out:", response)
    

    tokens = [t.strip() for t in response.split(",")]

    matched = []
    for token in tokens:
        closest = difflib.get_close_matches(token, valid_categories, n=1, cutoff=0.7)
        if closest and closest[0] not in matched:
            matched.append(closest[0])
        if len(matched) == 3:
            break

    if fallback_category not in matched:
      matched.append(fallback_category)
        
    return matched[:3]


Generate Answer

In [8]:
def generate_answer(query):
    categories = classify_query(query)
    print(f"Categories {categories}\n")
    relevant_documents = get_relevant_documents(query, categories)

    if not relevant_documents:
        return {
            "Response": "Sorry, no relevant documents found."
        }

    #relevant_documents = list(set(relevant_documents))

    seen = set()
    unique_docs = []
    for doc, sim in relevant_documents:
        if doc not in seen:
            seen.add(doc)
            unique_docs.append((doc, sim))

    print("Relevant Documents with Similarity Scores:")
    for doc, sim in unique_docs:
        print(f"Similarity: {sim:.4f}\nDoc: {doc}\n")

    relevant_texts = "\n\n".join([doc for doc, _ in unique_docs])
    
    ## additional prompts
    hike_prompt = f"""
        INSTRUCTIONS:
            1. Convert structured information about the hike into a short, friendly paragraph using natural language. Do not repeat numbers or use formatting from the source.
            2. If they ask about hiking information, only answer with required information. Users can ask for more information if needed.
            3. When asked for a particular type of hike, find it instead of saying that one would not work in the category they asked for.
            4. Do NOT list trail attributes or stats (like “Distance: 3.1 km, Elevation: 789 m”). Instead, describe them in context (e.g., “a steep 3 km trail with a tough 789 m climb”).
            5. Avoid repeating exact numbers unless essential (e.g., elevation gain is helpful, but don't dump all stats).
    """
    
    parks_prompt = f""" 
        INSTRUCTIONS:
            1. Convert structured information about the park into a short, friendly paragraph using natural language. Do not repeat numbers or use formatting from the source.
            2. Provide only necessary information that will allow the user to enjoy the park.
                - Feel free to tell them about logisitical information if asked.
    """
    
    food_prompt = f""" 
        INSTRUCTIONS:
            1. Convert structured food and dining information into a friendly, helpful paragraph. Do not copy the question or use list formatting.
            2. Only answer what the user asked. Do NOT add information that wasn't requested.
            3. Describe details in a natural way (e.g., “open 24/7 during the semester” instead of “Hours: 24/7”).
            4. Mention unique features only when they help clarify the user's question.
            5. If a specific venue or program is asked about (e.g., a café, meal plan, or food station), describe it clearly in context.
            6. If the question can't be answered from the data, respond with: “I'm sorry, I don't have that information. Please check the official SFU Food website.”
            7. Provide the official link when available and relevant to the answer.
            8. Do NOT list menu items, prices, or square footage unless directly relevant to the user's question.
            9. Only provide food information that is relevant. If they ask for some place that serves a chicken sandwich do not provide information to a vegan place.
    """

    ## activities general covers how to answer general parks, hikes, food, clubs, cultural related questions 
    activities_general = f""" 
        INSTRUCTIONS:
            1. If they ask for suggestions, provide 2 to 3 suggestions.
            2. Do NOT list all information. Instead describe them in context 
            3. Provide accuracte suggestions, NOT suggestions of things that will not work for what they want.
            4. Convert structured information about the activity into a short, friendly paragraph using natural language. Do not repeat formatting from the source.
    """
    
    ## permits prompt - covers ways to answer immigration, study permits, work permits, and permanent residence related questions 
    permits_prompt = f"""
        INSTRUCTIONS:
            1. When given a specific question with many possible answers, you can ask for more specific information.
                - if they are not asking for an extension do not provide information in regards to an extension of a permit.
            2. Only answer with information provided 
                - Information should NOT be guessed and do NOT add extra information
            3. If the answer is not in the dataset, respond with: "I'm sorry, I don't have that information. Please check the official IRCC website for more details."
            4. If it is helpful, provide the link and a description about it.
            5. Do NOT list all information. Instead describe them in context 
            6. If the answer depends on a specific condition explain those clearly.
            7. Do NOT make assumptions about the user's situation. 
    """
    
    housing_prompt = f""" 
        INSTRUCTIONS:
            1. Convert structured information about SFU or general student housing into a short, friendly paragraph using natural language. Do not repeat formatting or list prices unless helpful for context.
            2. Focus on what matters to the student: location, room types, meal plans, how to apply, and support available.
            3. Only mention costs in a general way (e.g., "starts around $4,000 per term") unless the user explicitly asks for detailed pricing.
            4. If information varies (e.g., by room type or campus), explain this clearly but briefly.
            5. If the user asks a specific housing question and the answer depends on certain conditions (e.g., term length, student status), explain those conditions clearly and simply.
            6. If the answer is not known or not in the data, respond with: "I’m sorry, I don’t have that information. Please check the SFU Housing website for details."
            7. Do NOT dump full lists of buildings, prices, or amenities. Summarize and keep it conversational.
            8. If the information is specific to SFU, make sure you say it to be clear.
    """
    
    transit_prompt = f""" 
        INSTRUCTIONS:
            1. Convert structured information about public transit into a short, friendly paragraph using natural language.
            2. Do NOT list statistics or technical formatting (like route numbers or fare charts) unless directly relevant to the user's question.
            3. Summarize relevant transit options clearly — describe them in context (e.g., “a quick SkyTrain ride from downtown to the airport”).
            4. Provide only what the user needs to understand how to get around or plan their trip.
            5. If the user is asking for directions, give a general summary of how they might travel.
            6. If the question is about fares, schedules, or route planning and the exact info is not available, tell the user to check the TransLink website and briefly explain what they can find there.
            7. Do NOT guess or make up transit information.
            8. If the information is not in the source, say “I’m sorry, I don’t have that information. You can check the official TransLink site for more details.”
    """

    
    ## main rag prompt - how to answer general questions 
    rag_prompt = f"""
    You are a helpful, friendly assistant for international students new to British Columbia, Canada.

    Below are some reference documents that may be relevant to the user's question:
    {relevant_texts}

    INSTRUCTIONS:
    1. If the user's query is just a greeting (like "hello", "hi", "what's up"):
       - Respond with a single brief friendly greeting
       - Offer to help with questions about studying or living in BC
       - Do NOT include ANY information from the reference documents
       - Do NOT create additional answers beyond answering their original question

    2. If the user is asking for information:
       - Be friendly and answer based ONLY on the reference documents if relevant
       - Summarize the necessary information into a couple sentences.
       - Do NOT create additional questions and answers beyond answering their original question
       - Limit your entire response to no more than 3 concise sentences when possible. Do not create long multi-line answers.
       - If the documents don't provide sufficient information, say "I don't have enough information to answer that. Please refer to official sources."
       - Ask for more information when there are multiple scenarios in the documents.
       - If they ask things like "can I", "will I", "how can I" feel free to ask follow up questions if you don't how to answer with the information provided. Do not just assume.
    
    3. IMPORTANT: Never generate additional content beyond answering the user's question. Do NOT number or bullet your points. Always use natural sentences and group similar information together where possible.
    
    User question: {query}

    Your response (just the answer, no preamble):
    """
    
    # adding the category specific prompting to main if necessary
    for category in categories:
        if category == "hiking" or category == "parks" or category == "food" or category == "cultural" or category == "clubs":
            rag_prompt += "\n" + activities_general
            
        if category == "hiking":
            rag_prompt += "\n" + hike_prompt
            
        if category == "parks":
            rag_prompt += "\n" + parks_prompt
        
        if category == "food":
            rag_prompt += "\n" + food_prompt
            
        if category == "study permit" or category == "work permit" or category == "immigration" or category == "permanent residence":
            rag_prompt += "\n" + permits_prompt
            
        if category == "housing":
            rag_prompt += "\n" + housing_prompt
        
        if category == "transit":
            rag_prompt += "\n" + transit_prompt
       
    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]
    # response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)

    return {
        "Response": response_after_rag
    }


Example Usage

In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

In [ ]:
answer = generate_answer("what if I only have one parent when applying for my study permit?")
print(answer["Response"])

In [10]:
import pandas as pd
from evaluate import load
bertscore = load("bertscore")

benchmark_data = pd.read_csv("../p-eval-set/seen-data.csv")

for idx, row in benchmark_data.iterrows():
    user_query = row["Question"]
    correct_answer = row["Answer"]

    responses = generate_answer(user_query)
    
    predictions = [responses.get("Response", "N/A")]
    references = [correct_answer]
    results = bertscore.compute(predictions=predictions, references=references, lang="en")

    print("\n" + "="*50)
    print(f"Benchmark Query {idx + 1}: {user_query}")
    print("="*50)
    print("\nRAG Response:\n", responses.get("Response", "N/A"))
    print("\n(Benchmark) Answer:\n", correct_answer)
    print("BERT Score == ", results)
    print("="*50 + "\n\n")


Raw out: transit, transportation, faq
Categories ['transit', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 0.7480
Doc: What type of public transit is available in BC? Most cities and towns in British Columbia have a public transit system. BC Transit is responsible for coordinating the different municipal transit systems in British Columbia, except in Metro Vancouver where it is done by TransLink. BC Transit provides bus services in over 50 communities across British Columbia. TransLink operates the SkyTrain and WestCoast Express (rail), SeaBus (boat) and bus services in Metro Vancouver. You can find local bus maps and schedules online. For more information: BC Transit: Choose a Transit System BC Transit: Choose a Transit System TransLink: Schedules and Maps TransLink: Schedules and Maps



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Benchmark Query 1: What type of public transportation is available in Vancouver?

RAG Response:
 
    User question: What type of public transportation is available in Vancouver?

    Your response:
    
    Vancouver has a comprehensive public transportation system, operated by TransLink. The main modes of transportation include buses, the SkyTrain (rapid transit), the WestCoast Express (commuter rail), the SeaBus (water ferry), and HandyDart (paratransit). The SkyTrain and buses cover the urban areas, while the WestCoast Express and SeaBus serve specific routes. You can find schedules, maps, and more detailed information on the TransLink website.

(Benchmark) Answer:
 Vancouver has several types of public transportation: Buses - includes regular buses, express buses (marked with 'X'), RapidBus routes (marked with 'R'), and NightBus service (routes starting with 'N'). SkyTrain - includes the Expo Line which connects downtown Vancouver, Burnaby, and Surrey, the Canada Line which conne